In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(16,8))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack.qubicfp import qubicfp
from qubic import fibtools as ft
from qubic import plotters as p
from qubic import lin_lib as ll
from qubic import demodulation_lib as dl
from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
from importlib import reload  


## Dataset
Lets get the directories corresponding to each dataset:

In [ ]:
#### All OK
# n = 'ScanFreq'
# days = ['2019-04-12']

savedir = '/Users/hamilton/Qubic/Calib-TD/BandPass/'



#### Only take from 80 to the end
n = 'ScanFreq'
datadir = '/Volumes/HD JC-Hamilton/QubicData/Calib-TD/'
days = ['2019-04-15', '2019-04-16']

dirs = []
freqs=[]
for d in days:
    dd = glob.glob(datadir+'/'+d+'/*'+n+'*')
    for i in range(len(dd)): 
        #print(dd[i])
        truc = str.split(dd[i],'_')
        the_nu = truc[-1]
        try:
            nufloat = float(the_nu)
            freqs.append(float(the_nu))
            dirs.append(dd[i])
        except:
            print('File '+dd[i]+' has a format that des not comply with expectations => rejected')
            continue
    print('  * ',d,' : {} files'.format(len(dd)))
print('  => Total = {} files'.format(len(dirs)))
print('  => Frequency ranges from {} to {}'.format(np.min(freqs),np.max(freqs)))

dirs = dirs[80:]
freqs = freqs[80:]
print('After cut')
print('  => Total = {} files'.format(len(dirs)))
print('  => Frequency ranges from {} to {}'.format(np.min(freqs),np.max(freqs)))


In [ ]:
labels = []
dir_time = []
for d in dirs:
    bla = str.split(d,'__')
    blo = str.split(bla[0],'/')
    labels.append(bla[1])
    dir_time.append(blo[-1])
    
for i in range(len(labels)): 
    print(i, labels[i], dir_time[i], 'Frequency: ', freqs[i])


# Run demodulation

In [ ]:
ii = 0

reload(dl)
reload(ft)

#### Parameters
freq_mod = 1.
ppp = 1./freq_mod
lowcut = 0.3
highcut = 10.
method = 'rms'
# method = 'demod'

allnums = np.arange(len(dirs))
nbtodo = len(dirs)
mynums = allnums[(allnums % (len(allnums)//nbtodo)) == 0]
print()
print('##############################################################')
print('There are {} files to analyze'.format(len(allnums)))
print('I was asked to do {} of them, so doing numbers {}'.format(nbtodo, mynums))
print('##############################################################')
print()
print()

alltime = []
alldemodulated = []
allerrdemodulated = []
allfreqs = []

for ii in range(len(mynums)):
    thedir = dirs[mynums[ii]]
    thefreq = freqs[mynums[ii]]
    print('##############################################################')
    print('Directory {} / {} :'.format(ii, len(mynums)), thedir)
    print('##############################################################')
    print('Not Done yet: doing it...')
    a = qubicfp()
    a.read_qubicstudio_dataset(thedir)
    data = a.azel_etc()

    mytt = []
    mydemod = []
    myerrdemod = []
    for iasic in [0,1]:
        print('======== ASIC {} ====================='.format(iasic))
        AsicNum = iasic+1
        mydata = [data['t_data %i' % AsicNum], data['data %i' % AsicNum]]
        if data['t_src'] is not None: 
            src_data = [data['t_src'], data['data_src']]
        else:
            src_data = None
        #if data['t_src'] is not None: data['t_src'] += 7200

        ### New Code
        mytime, demodulated, errdemodulated = dl.demodulate_methods(mydata, freq_mod, 
                                                                    fourier_cuts=[lowcut, highcut, None], 
                                                                    src_data_in = src_data,
                                                                   verbose=False, method=method, remove_noise=True)
        mydemod.append(demodulated)
        myerrdemod.append(errdemodulated)
    alldemodulated.append(np.append(mydemod[0], mydemod[1], axis=0))
    allerrdemodulated.append(np.append(myerrdemod[0], myerrdemod[1], axis=0))
    allfreqs.append(thefreq)

    


In [ ]:
allnbins = np.zeros(len(freqs), dtype=int)
for i in range(len(freqs)):
    allnbins[i] = alldemodulated[i].shape[1]
    
mynbins = np.max(allnbins)

my_alldemodulated = np.zeros((len(freqs), 256, mynbins))
my_allerrdemodulated = np.zeros((len(freqs), 256, mynbins))
for i in range(len(freqs)):
    sh = np.shape(alldemodulated[i])
    if sh[1]==mynbins:
        my_alldemodulated[i,:,:] = alldemodulated[i]
        my_allerrdemodulated[i,:,:] = allerrdemodulated[i]
    else:
        print(sh[1])
        my_alldemodulated[i,:,0:sh[1]] = alldemodulated[i]        
        my_allerrdemodulated[i,:,0:sh[1]] = allerrdemodulated[i]

name = method+'_'+n
for d in days:
    name = name+'_'+d

FitsArray(my_alldemodulated).save(savedir+'/alldemodulated_'+name+'.fits')
FitsArray(my_allerrdemodulated).save(savedir+'/allerrdemodulated_'+name+'.fits')
FitsArray(allfreqs).save(savedir+'/allfreqs_'+name+'.fits')


# Now read files and average demodulated data

In [ ]:
method = 'rms'
name = method+'_'+n
for d in days:
    name = name+'_'+d

alldemodulated = FitsArray(savedir+'/alldemodulated_'+name+'.fits')
allerrdemodulated = FitsArray(savedir+'/allerrdemodulated_'+name+'.fits')
allfreqs = FitsArray(savedir+'/allfreqs_'+name+'.fits')


print(alldemodulated.shape)
print(allerrdemodulated.shape)
print(allfreqs.shape)


allmm = np.zeros((len(mynums), 256))
allss = np.zeros((len(mynums), 256))

for j in range(len(mynums)):
    if (j % 10)==0: print('{} over {}'.format(j,len(mynums)-1))
    for i in range(256):
        allmm[j, i], allss[j, i] = ft.meancut(alldemodulated[j,i,:], 3, med=True, disp=False)



In [ ]:
reload(dl)

ff, f_rms, errf_rms = dl.get_spectral_response(name, allfreqs, allmm.T, allss.T, 
                                               method='rms',directory=savedir, correct_source=False)



figure()
errorbar(ff, dl.dB(f_rms), yerr=dl.dB(f_rms+errf_rms)-dl.dB(f_rms),label='rms', fmt='b.-',ls='none')
legend()
ylabel('dB')
xlabel('Frequency [GHz]')
title(name)
ylim(-40,0)
sorted_index=sorted(range(len(ff)), key=lambda i: ff[i])
plot(ff[sorted_index],dl.dB(f_rms[sorted_index]))



In [ ]:
rff, rf_rms, drff, rerrf_rms, bla = ft.profile(ff, f_rms, nbins=64, plot=False, dispersion=True)


rc('figure',figsize=(16,8))
rc('font',size=18)
figure()
errorbar(rff, dl.dB(rf_rms), yerr=dl.dB(rf_rms+rerrf_rms)-dl.dB(rf_rms),label='rms', fmt='bo',ls='none')
legend()
ylabel('dB')
xlabel('Frequency [GHz]')
#title(name)
ylim(-25,1)
sorted_index=sorted(range(len(rff)), key=lambda i: rff[i])
plot(rff[sorted_index],dl.dB(rf_rms[sorted_index]))
axhline(y=0, color='k',ls=':')
savefig(savedir+'bandpass_qubic.pdf')